In [1]:
import pandas as pd 
import json
from arcgis.features import FeatureLayer
import requests
import time

/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# API Call to Louisville Open Data. 

| Steps      | Description | Notes     |
| :---        |    :----   |          :--- |
| 1. Open Data API call      | API call for restaurant inspection scores | Works   |
| 2. Clean   | Remove un-needed data        | drop cols and sort data. Also ned to only keep the most recent scores      |

In [2]:
url = 'https://services1.arcgis.com/79kfd2K6fskCAkyg/arcgis/rest/services/FoodServiceData/FeatureServer/0'

batch_size = 1000  # Number of records to retrieve per batch
offset = 0  # Initial offset value
data_list = []
# ​Create the feature layer object
feature_layer = FeatureLayer(url)

while True:
    # Query the feature layer with pagination
    query_result = feature_layer.query(where='1=1', out_fields='*', return_geometry=False, result_offset=offset, result_record_count=batch_size)
    
    # Retrieve the features from the query result
    features = query_result.features
    
    # Process the data for the current batch
    for feature in features:
        data_list.append(feature.attributes)
    
    # Break the loop if the response is empty or the desired number of records is reached
    if len(features) == 0 or len(data_list) >= 1000:
        break
    
    # Increment the offset by the batch size
    offset += batch_size
# Create a DataFrame from the data list
df = pd.DataFrame(data_list)

Verify the data comes in correct

In [3]:
df.head()

,EstablishmentID,InspectionID,Ins_TypeDesc,EstablishmentName,PlaceName,Address,Address2,City,State,Zip,TypeDescription,InspectionDate,score,Grade,NameSearch,Intersection,ObjectId
0,29956,1768584,REGULAR,CVS #6211,None,3721 LEXINGTON RD,None,LOUISVILLE,KY,40207,PRE-PACKAGED RETAIL,2023-06-30 00:00:00,100,None,CVS #6211,None,1
1,29967,1786484,REGULAR,NEIGHBORHOOD FOOD MART,None,542 LAMPTON ST,None,LOUISVILLE,KY,40203,SUPERMARKET WITH PROCESSING,2023-09-05 00:00:00,100,None,NEIGHBORHOOD FOOD MART,None,2
2,29979,1731827,REGULAR,CIRCLE K #4700075,None,1630 ARTHUR ST,None,LOUISVILLE,KY,40208,"RETAIL-FOOD 10,000 SQ FT OR LESS",2023-01-27 00:00:00,95,A,CIRCLE K #4700075,None,3
3,29979,1731828,REGULAR,CIRCLE K #4700075,None,1630 ARTHUR ST,None,LOUISVILLE,KY,40208,"RETAIL-FOOD 10,000 SQ FT OR LESS",2023-01-27 00:00:00,97,None,CIRCLE K #4700075,None,4
4,29979,1766455,REGULAR,CIRCLE K #4700075,None,1630 ARTHUR ST,None,LOUISVILLE,KY,40208,"RETAIL-FOOD 10,000 SQ FT OR LESS",2023-06-21 00:00:00,98,A,CIRCLE K #4700075,None,5


Drops the extra cols we dont need. 

In [4]:
cols_drop = ['EstablishmentID', 'InspectionID', 'PlaceName', 'Address2', 'TypeDescription', 'NameSearch', 'Intersection']
df.drop(cols_drop, axis=1, inplace=True)
df.head(2)

,Ins_TypeDesc,EstablishmentName,Address,City,State,Zip,InspectionDate,score,Grade,ObjectId
0,REGULAR,CVS #6211,3721 LEXINGTON RD,LOUISVILLE,KY,40207,2023-06-30 00:00:00,100,None,1
1,REGULAR,NEIGHBORHOOD FOOD MART,542 LAMPTON ST,LOUISVILLE,KY,40203,2023-09-05 00:00:00,100,None,2


In [5]:
df.shape

(11036, 10)

Converting InspectionDate to date time

In [6]:
df['InspectionDate'] = pd.to_datetime(df['InspectionDate'])
# sorting inspection dates
df.sort_values('InspectionDate', ascending=False, inplace=True)
# dropping duplicate rest based on its first occurrence
df.drop_duplicates(subset='EstablishmentName', keep='first', inplace=True)
df.head(2)

,Ins_TypeDesc,EstablishmentName,Address,City,State,Zip,InspectionDate,score,Grade,ObjectId
8321,REGULAR,HABESHA GROCERY & RESTAURANT,7136 SOUTHSIDE DR,LOUISVILLE,KY,40214,2023-09-15,96,A,8322
690,REGULAR,SPEEDWAY #9645,7400 OLD 3RD STREET RD,LOUISVILLE,KY,40214,2023-09-15,99,A,691


In [7]:
df.shape

(4108, 10)

In [8]:
df.value_counts('EstablishmentName')

EstablishmentName
#1 CHINESE RESTAURANT         1
PRICE ELEMENTARY SCHOOL       1
PENDENNIS CLUB BAR            1
PENN STATION                  1
PENN STATION #5               1
                             ..
FROZEN DEE LITES              1
FROZEN FLAKES                 1
FRY DADDY'S                   1
FRY FACTORY                   1
ZOO WILD BURGER LOUISVILLE    1
Length: 4108, dtype: int64

In [9]:
df = df.sort_values(by='score', ascending=True)
df.head(3)

,Ins_TypeDesc,EstablishmentName,Address,City,State,Zip,InspectionDate,score,Grade,ObjectId
10211,FOLLOWUP,MISS MARY'S CONVENIENCE STORE,500 S 20TH ST,LOUISVILLE,KY,40203,2023-07-14,0,None,10212
6730,FOLLOWUP,FAMILY TIES,1030 CECIL AVE,LOUISVILLE,KY,40211,2023-06-22,0,None,6731
8654,FOLLOWUP,SAGARMATHA GROCERY,4010 BARDSTOWN RD,LOUISVILLE,KY,40218,2023-04-13,0,None,8655


In [10]:
df.to_json('../json_files/health.json')

# Yelp API 

Imports my API key for yelp. 

In [11]:
from akeys import api_key

In [22]:
api_key
endpoint = "businesses/search"
url = f"https://api.yelp.com/v3/{endpoint}"
headers = {
    "Authorization": f"Bearer {api_key}"
}
params = {
    "term": "restaurants",
    "location": "Louisville, KY",
    "limit": 40  # Set the desired limit per request (maximum is 50)
}

restaurant_data = []  # List to store the extracted data

offset = 0  # Initial offset value
results_per_request = params["limit"]  # Results per request (50)
total_results = float("inf")  # Initialize total_results to an arbitrary high value

while offset < total_results:
    params["offset"] = offset  # Set the offset parameter
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code} - {response.text}")
        break
    
    data = response.json()

    if offset == 0:
        total_results = data["total"]
    
    businesses = data["businesses"]
    
    for business in businesses:
        name = business.get("name")
        rating = business.get("rating")
        review_count = business.get("review_count")
        price = business.get("price")
        address = ", ".join(business.get("location", {}).get("display_address", []))
        restaurant_data.append({
            "Name": name,
            "Rating": rating,
            "Review Count": review_count,
            "Price": price,
            "Address": address
        })
    offset += results_per_request  
    time.sleep(2)  

# Create a DataFrame from the extracted data
df2 = pd.DataFrame(restaurant_data)

# Print the DataFrame
df2.head(3)

Error: 400 - {"error": {"code": "VALIDATION_ERROR", "description": "Too many results requested, limit+offset must be <= 1000."}}


,Name,Rating,Review Count,Price,Address
0,The Eagle,4.5,1562,$$,"1314 Bardstown Rd, Louisville, KY 40204"
1,Hammerheads,4.5,1338,$$,"921 Swan St, Louisville, KY 40204"
2,Oliver's Chop House & Bourbon Bar,4.5,11,None,"4520 Poplar Level Rd, Derby City Gaming Hotel,..."


In [27]:
df2.shape
df2.to_csv('yelp.csv')

In [29]:
df = pd.read_csv('json_files/local_working.csv')
df2 = pd.read_csv('json_files/yelp.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'json_files/local_working.csv'

In [20]:
# # cols_drop = ['InspectionID', 'PlaceName', 'Address2', 'TypeDescription', 'NameSearch', 'Intersection']
# # df.drop(cols_drop, axis=1, inplace=True)
# df['InspectionDate'] = pd.to_datetime(df['InspectionDate'])
# # Grouping by EstablishmentID and aggregating the necessary columns
# df = df.groupby("EstablishmentID").agg({
#     "EstablishmentName": "first",
#     "Address": "first",
#     "score": lambda x: list(x)
# })

# # Sorting the establishments based on the count of low scores
# df["low_score_count"] = df["score"].apply(lambda x: sum(score < 75 for score in x))
# df = df.sort_values("low_score_count", ascending=False)

# # Creating the final DataFrame with the desired columns
# df = df[["EstablishmentName", "Address", "score"]]

# # Printing the final DataFrame
# df.head(10)

In [19]:
df2 = df2.sort_values(by="Rating", ascending=True)
df2 = df2.loc[df['Rating'] != 0.0]
df2 = df2.reset_index(drop=True)
df2['Rating'] = (df2['Rating']*20).astype(int)
df2.head(3)
df2.to_json('../json_files/yelp.json')

KeyError: 'Rating'

In [ ]:
df2['Address'] = df2['Address'].str.split(',').str[0]
df2['Rating'] = (df2['Rating']*20).astype(int)
df2.head(3)

In [ ]:
df['Address'] = df['Address'].str.lower().str.strip()
df2['Address'] = df2['Address'].str.lower().str.strip()

In [ ]:
merged_df = df.merge(df2, on='Address', how='left')
merged_df = merged_df.sort_values('Rating', ascending=True)
merged_df = merged_df[['EstablishmentName', 'Address', 'score', 'Rating', 'Review Count']]
merged_df.head(3)

In [ ]:
processed_data = merged_df.to_json(orient='records')

In [ ]:
# Save the processed data as JSON to a file
with open('json_files/processed_data.json', 'w') as file:
    json.dump(processed_data, file)